In [ ]:
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import firedrake
from firedrake import assemble, Constant, min_value, max_value, inner, grad, dx, ds
import icepack
from icepack.constants import ice_density as ρ_I, gravity as g

In [ ]:
filename = "spin_up_state.h5"
with firedrake.CheckpointFile(filename, "r") as chk:
    mesh = chk.load_mesh()
    C = chk.load_function(mesh, "friction")
    h_0 = chk.load_function(mesh, "thickness")
    s_0 = chk.load_function(mesh, "surface")
    u_0 = chk.load_function(mesh, "velocity")
    
Q = h_0.function_space()
V = u_0.function_space()

In [ ]:
coords = mesh.coordinates.dat.data_ro[:]

xmin, xmax = coords[:, 0].min(), coords[:, 0].max()
ymin, ymax = coords[:, 1].min(), coords[:, 1].max()

In [ ]:
delta = 10e3

image_filename = icepack.datasets.fetch_mosaic_of_antarctica()
with rasterio.open(image_filename, "r") as image_file:
    height, width = image_file.height, image_file.width
    transform = image_file.transform
    window = rasterio.windows.from_bounds(
        left=xmin - delta,
        bottom=ymin - delta,
        right=xmax + delta,
        top=ymax + delta,
        transform=transform,
    )
    image = image_file.read(indexes=1, window=window, masked=True)

def subplots(*args, **kwargs):
    fig, axes = plt.subplots(*args, **kwargs)
    axes.set_aspect("equal")
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(window, transform)
    axes.imshow(
        image,
        cmap="Greys_r",
        vmin=12e3,
        vmax=16.38e3,
        extent=(xmin, xmax, ymin, ymax),
    )

    return fig, axes

In [ ]:
fig, axes = subplots()
firedrake.triplot(mesh, axes=axes)
axes.legend();

In [ ]:
Q = h_0.function_space()
V = u_0.function_space()

In [ ]:
b = firedrake.interpolate(s_0 - h_0, Q)

In [ ]:
def terminus(**kwargs):
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    s = kwargs["surface"]
    h_D = kwargs["thickness_downstream"]
    
    τ_I = 0.5 * ρ_I * g * h**2
    τ_D = 0.5 * ρ_I * g * h_D**2

    ν = firedrake.FacetNormal(mesh)
    return (τ_I - τ_D) * inner(u, ν)

In [ ]:
model = icepack.models.IceStream(terminus=terminus)
opts = {
    "dirichlet_ids": [1, 2, 3, 5, 6, 7],
    "diagnostic_solver_type": "petsc",
    "diagnostic_solver_parameters": {
        "snes_max_it": 100,
        "snes_type": "newtontr",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}
solver = icepack.solvers.FlowSolver(model, **opts)

In [ ]:
h_D = h_0.copy(deepcopy=True)
u = u_0.copy(deepcopy=True)

What we're going to do for our perturbation experiment is pick a dimensionless shape function $\phi$ that goes from 0 on all the other boundaries to 1 on the inflow boundary.
We'll then set the inflow thickness to $h + \delta h\cdot\phi$, run it for a while, and see what happens for different values of $\delta h$.
The shape function $\phi$ should be as smooth as possible; to define it, we'll let it be the solution of the PDE

$$-\alpha^2\nabla^2\phi = 1$$

subject to the condition that $\phi = 0$ on all the outflow, side wall, and nunatak boundaries, and $\nabla\phi\cdot\nu = 0$ on the inflow boundary.
(The vector $\nu$ is the unit outward-pointing normal vector to the boundary of the domain.)
We can get other shapes by putting something other than 1 on the right-hand side and I can show you how to do that later; this is just for illustrative purposes.

In [ ]:
ϕ = firedrake.Function(Q)

f = Constant(1.0)
α = Constant(2e3)
J = (0.5 * α**2 * inner(grad(ϕ), grad(ϕ)) - f * ϕ) * dx
F = firedrake.derivative(J, ϕ)
boundary_ids = (1, 3, 4, 5, 6, 7)
bc = firedrake.DirichletBC(Q, 0, boundary_ids)
firedrake.solve(F == 0, ϕ, bc)

ϕ_max = ϕ.dat.data_ro.max()
ϕ /= ϕ_max

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(ϕ, axes=axes)
fig.colorbar(colors);

To decide how much of a perturbation we want to make, we'll first evaluate the average thickness along the inflow boundary.

In [ ]:
inflow_ids = (2,)
inflow_length = assemble(Constant(1, domain=mesh) * ds(inflow_ids))
avg_inflow_thickness = assemble(h_0 * ds(inflow_ids)) / inflow_length
print(f"Average inflow thickness: {avg_inflow_thickness:5.1f} m")

Let's assume the maximum perturbation amplitude is 1/20 of the average.

In [ ]:
δh = Constant(avg_inflow_thickness / 20)
h = firedrake.interpolate(h_0 + δh * ϕ, Q)
h_inflow = h.copy(deepcopy=True)

We'll need to recompute the surface height too.

In [ ]:
s = icepack.compute_surface(thickness=h, bed=b)

Now let's run the model forward and see what happens.

In [ ]:
T = Constant(260.0)
A = icepack.rate_factor(T)
a = Constant(0.1)

In [ ]:
from firedrake import min_value, max_value

s_min = Constant(np.floor(s_0.dat.data_ro.min()))
s_max = Constant(np.ceil(s_0.dat.data_ro.max()))

a_min = Constant(0.06)
a_max = Constant(0.10)

def accumulation(s):
    λ = (s - s_min) / (s_max - s_min)
    a = (1 - λ) * a_min + λ * a_max
    return min_value(a_max, max_value(a_min, a))

In [ ]:
hs = [h.copy(deepcopy=True)]
ss = [s.copy(deepcopy=True)]
us = [u.copy(deepcopy=True)]

In [ ]:
final_time = 1200.0
dt = 1 / 2
num_steps = int(final_time / dt)

for step in tqdm.trange(num_steps):
    a = firedrake.interpolate(accumulation(s), Q)
    
    h = solver.prognostic_solve(
        dt,
        thickness=h,
        velocity=u,
        accumulation=a,
        thickness_inflow=h_inflow,
    )
    h.interpolate(firedrake.max_value(1.0, h))

    s = icepack.compute_surface(thickness=h, bed=b)

    u = solver.diagnostic_solve(
        velocity=u,
        thickness=h,
        surface=s,
        fluidity=A,
        friction=C,
        thickness_downstream=h_D,
    )
    
    hs.append(h.copy(deepcopy=True))
    ss.append(s.copy(deepcopy=True))
    us.append(u.copy(deepcopy=True))

In [ ]:
δhs = [firedrake.interpolate(h - h_0, Q) for h in hs]

In [ ]:
δh_min = np.floor(np.array([δh.dat.data_ro.min() for δh in δhs]).min())
δh_max = np.ceil(np.array([δh.dat.data_ro.max() for δh in δhs]).max())
δh_min, δh_max

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation

fig, axes = subplots()
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)

colors = firedrake.tripcolor(
    δhs[0], axes=axes, vmin=0.0, vmax=δh_max / 2, cmap="Blues", num_sample_points=4
)
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)

def animate(δh):
    colors.set_array(fn_plotter(δh))

interval = 1e3 / 60
animation = FuncAnimation(fig, animate, frames=δhs, interval=interval)

In [ ]:
from IPython.display import HTML

HTML(animation.to_html5_video())

The simulation before demonstrated one way to recompute the accumulation rate as a function of the surface elevation at every step.
Here we'll reverse the perturbation from the inflow thickness slowly over time.

In [ ]:
final_time = 1200.0
dt = 1 / 2
num_steps = int(final_time / dt)

λ = Constant(0.0)

for step in tqdm.trange(num_steps):
    a = firedrake.interpolate(accumulation(s), Q)
    
    λ.assign(min(step * dt / (final_time / 4), 1.0))
    δh.assign((1 - λ) * avg_inflow_thickness / 20)
    h_inflow.interpolate(h_0 + δh * ϕ)
    
    h = solver.prognostic_solve(
        dt,
        thickness=h,
        velocity=u,
        accumulation=a,
        thickness_inflow=h_inflow,
    )
    h.interpolate(firedrake.max_value(1.0, h))

    s = icepack.compute_surface(thickness=h, bed=b)

    u = solver.diagnostic_solve(
        velocity=u,
        thickness=h,
        surface=s,
        fluidity=A,
        friction=C,
        thickness_downstream=h_D,
    )
    
    hs.append(h.copy(deepcopy=True))
    ss.append(s.copy(deepcopy=True))
    us.append(u.copy(deepcopy=True))

Finally, we'll plot the average thickness throughout the whole domain as a function of time.
The first 1200 years show increase, the second 1200 relaxation back.

In [ ]:
area = assemble(Constant(1) * dx(mesh))
avg_thicknesses = [assemble(h * dx) / area for h in hs]

In [ ]:
fig, axes = plt.subplots()
ts = np.linspace(0.0, 2 * final_time, 2 * num_steps + 1)
axes.set_xlabel("time (years)")
axes.set_ylabel("average thickness (meters)")
axes.plot(ts, avg_thicknesses);